# Exercícios de Introdução
Aluno: Marco Antonio Reche Rigon

Vamos começar importando as bibliotecas que serão usadas ao decorrer dos exercícios

In [ ]:
import cv2 as cv
import numpy as np

Como estamos mexendo com arquivos, é interessante fazer uma verificação para garantir que os arquivos foram lidos com sucesso.

In [ ]:
import os
imagesPath = 'images/'

def buscaImagens(imgs: dict[str,str]) -> dict[str, np.ndarray]:
    images: dict[str, np.ndarray] = {}

    for name, filename in imgs.items():
        fullPath = os.path.join(imagesPath, filename)
        
        if os.path.exists(fullPath):
            img = cv.imread(fullPath)
            if img is not None:
                images[name] = img
            else:
                print(f"Erro: Não foi possível carregar a imagem '{filename}'. O arquivo pode estar corrompido.")
        else:
            print(f"Erro: O arquivo '{filename}' não foi encontrado em '{imagesPath}'.")
    return images


## Ex. 01
Nesse exercício, iremos usar três imagens com variação de cinza. Os tons de cinza são representados por pixels de 8 bits. Ou seja, entre 0 e 255.

In [3]:
imgs = buscaImagens({
    'clara': 'figuraClara.jpg',
    'cinza': 'figuraCinza.jpg',
    'escura': 'figuraEscura.jpg'
})

Vamos fazer um algoritmo para exibir as imagens.

In [4]:
cv.imshow('Exercicio 01', imgs['cinza'])
cv.waitKey(0)

cv.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


Uau, quem diria que a imagem cinza é bem cinza... O quão cinza essa imagem é? Vamos fazer uma média de cores para ver isso. Para operações matriciais é muito útil usar o Numpy, que já vem com muitas funções embutidas para realizar essas operações. Tão útil que o OpenCV usa matrizes Numpy para representar suas imagens.

In [5]:
print(np.mean(imgs['cinza']))
print(imgs['cinza'][0][0])

130.1064531069319
[117 117 117]


## Ex. 02


Numpy é muito legal, mas vamos implementar a média na mão? OpenCV recomenda usarmos funções Numpy para isso. E tanto o OpenCV, quanto Numpy recomendam não fazer operações acessando elementos individualmente. Para fins didáticos, vamos quebrar as regras.

In [ ]:
def media(img: np.ndarray) -> float:
    count = 0
    acc = 0.0 # float64 para evitar overflow
    for i in range(len(img)):
        for j in range(len(img[i])):
            for k in range(len(img[i][j])):
                acc += img[i][j][k]
                count += 1
    return acc / count

print(media(imgs['cinza']))

130.1064531069319


Como eu disse, essa função é bem lenta por estarmos acessando diversos valores individuais com Numpy (não faça leitura de matrizes Numpy percorrendo ela inteira). Vamos continuar usando `np.mean()` porque não quero perder tempo esperando a função terminar de rodar.

### Escuridão

Vamos achar um jeito de identificar o quão escuro é uma imagem. Antes de pensar numa solução, vamos analisar a média dos valores das imagens que temos.

In [8]:
print(np.mean(imgs['clara']))
print(np.mean(imgs['cinza']))
print(np.mean(imgs['escura']))

252.06378538951932
130.1064531069319
15.06361587097217


Podemos ver que quanto mais escuro a imagem, menor será a média dos valores. Afinal, `(255,255,255)` é branco e `(0,0,0)` é preto. Mas temos um problema, isso é só a média dos valores. Não implica que a imagem é muito clara, cinza ou escura. E se nós fizermos essa comparação para média para cada canal (canais são RGB) e ver se elas são muito próximas? Tendo isso em mente, vamos implementar essa função.

In [ ]:

def nivelEscuro(img: np.ndarray, delta = 10) -> int:
    '''
        Função que verifica o nível de escuridão de uma image
        Argumentos:
            img: Imagem que será verificada
            delta: Variação permitida
        Retorno
            Inteiro com o nível de escuridão da imagem:
                -1 Normal
                 0 Muito escuro
                 1 Muito cinza
                 2 Muito claro
    '''
    r = np.mean(img[:, :, 2]) # OpenCV carrega imagens como BGR ao invés de RGB
    g = np.mean(img[:, :, 1]) # Os dois pontos quer dizer "Pegue tudo desse eixo"
    b = np.mean(img[:, :, 0])
    rgb = np.mean([r,g,b])
    var = np.var([r,g,b])

    if -delta < var < delta:
        if rgb < 86:
            return 0
        if rgb > 172:
            return 2
        return 1
    return -1

print(nivelEscuro(cv.imread('images/arco_iris.jpg'))) # type: ignore
print(nivelEscuro(imgs['escura']))
print(nivelEscuro(imgs['cinza']))
print(nivelEscuro(imgs['clara']))

-1
0
1
2
